## Overview
---
Our illustrative MD example is an analytically tractable microscopic MD model of Brownian motion which was formulated in Notebook(8.4) in terms of a (heavy) diffusing particle (described as a ball $B$ with radius $R$ and mass $m_{p}$ ) colliding with a large number of (light) point solvent particles with masses $m_{s} \ll m_{p} .$ Denoting the coordinates of the centre of the heavy particle by $\mathbf{X}=\left[X_{1}, X_{2}, X_{3}\right]$ and its velocity by $\mathbf{V}=\left[V_{1}, V_{2}, V_{3}\right],$ we showed that they converge to the solution of the Langevin equation in the limit $m_{p} / m_{s} \rightarrow \infty,$ provided that the density and velocity distributions of solvent particles are scaled accordingly.

## Initial Conditions
---
We consider that our computational domain is the cube with the length of edge as $L$. Since the diffusing particle is the most important part of our simulation, we initially place it in the middle of our computational domain $\mathbf{X}(0)=[L / 2, L / 2, L / 2]$.

Assuming that $\Omega$ contains just one diffusing particle $-$ the ball $B-$ the number of solvent particles in $\Omega$ is distributed according to the Poisson distribution with mean $\lambda_{\mu}|\Omega \backslash B|,$ where $|\Omega \backslash B|$ is the volume of the space available to solvent particles (i.e. the computational domain $\Omega$ without the ball $B$ ) and the constant $\lambda_{\mu}$ is given by:
$$
\lambda_{\mu}=\frac{3}{8 R^{2}} \sqrt{\frac{(\mu+1) \beta}{2 \pi D}}
$$

To initialize the positions and velocities of the solvent molecules, we first sample a random number $N_{a}$ from the Poisson distribution with mean $A_{\mu} L^{3}$ and attempt to place $N_{a}$ molecules at uniformly distributed positions inside the cube $\Omega .$ This can be done by sampling three independent uniformly distributed random numbers $r_{1}, r_{2}$ and $r_{3}$ in interval $[0,1]$ for each solvent molecule and choosing its position in $\Omega$ as $x^{i}=L\left[r_{1}, r_{2}, r_{3}\right] .$ Then solvent molecules in $\Omega \backslash B$ are initially distributed according to the (spatial) Poisson distribution with density $\lambda_{\mu} .$ 

We also have to initialize their velocities according to the Maxwell distribution shown in the notebook (8.4). This can be done by sampling three independent normally distributed (with zero mean and unit variance) random numbers $\xi_i$,$\xi_2$, and $\xi_3$ for each solvent molecule and choosing its intial velocity as $v^i=\sigma_{\mu}[\xi_1,\xi_2,\xi_3]$, where $\sigma_{\mu}$ is given in the notebook (8.4).

## Time Evolution Equations
---
If we evolve time using a fixed time step $\Delta t$, then the evolution of the positions of molecules during one time step can be updated by the  "**free-flight**" equations:

$$
\begin{array}{l}{\mathbf{X}(t+\Delta t)=\mathbf{X}(t)+\mathbf{V}(t) \Delta t}\text{---(1)} \\ {\mathbf{x}^{i}(t+\Delta t)=\mathbf{x}^{i}(t)+\mathbf{v}^{i}(t) \Delta t, \quad \text { for } i=1,2, \ldots, N(t)}\text{---(2)}\end{array}
$$

where $N(t)$ is the total number of solvent molecules at time $t$. If a solvent molecule **does not collide with a large molecule, then the "free-fight" equation provides the correct update rule for its position and its velocity does not change during the time step**. Otherwise, the velocity is updated according to (1) in notebook (8.4).

The time evolution of the system can then be simulated by repeating the following three steps:

>1. Calculate the "free-flight" positions of the heavy particle, $\mathbf{X}(t+\Delta t),$
and solvent molecules, $\mathbf{x}^{i}(t+\Delta t),$ using $(1)-(2) .$

>2. If there exists $j \in\{1,2, \ldots, N(t)\}$ such that $\left|\mathbf{X}(t+\Delta t)-\mathbf{x}^{j}(t+\Delta t)\right|<R,$ then implement collisions of molecules according to $(1) in (8.4).$

>3. Implement suitable boundary conditions on the boundary of $\Omega,$ then continue with step $(a 23)$ for time $t+\Delta t$

## Langevin Description of Heat Bath particles outside $\Omega$
---
The diffusing molecule is the ball with centre $\mathbf{X}=\left[X_{1}, X_{2}, X_{3}\right]$ with velocity $\mathbf{v}=\left[V_{1}, V_{2}, V_{3}\right]$ and radius $R .$ It intersects interface $\left\{x_{1}=0\right\}$ if $X_{1}(t) \in(-R, R) .$ Heat bath particles in $\Omega$ are simulated using the MD model. We do not
explicitly simulate the heat bath particles outside $\Omega,$ but **model their effect by an additional correction to the velocity of the diffusing molecule in the form:**

$$
V_{i}(t+\Delta t)=\widetilde{V}_{i}(t+\Delta t)+f_{i}(t) \Delta t+g_{i}(t) \sqrt{\Delta t} \xi_{i}, \quad i=1,2,3\text{--(3)}
$$

where $\Delta t$ is a small time step and $\tilde{\mathbf{V}}(t+\Delta t)$ is the post-collision velocity of the diffusing molecule at time $t+\Delta t,$ which takes into account only collisions with the heat bath particles from inside $\Omega .$

Following the same approach as in notebook (8.4) we find:

$$
\begin{aligned} f_{1}(t) &=-h_{1}\left(X_{1}(t)\right) \beta V_{1}(t)+h_{0}\left(X_{1}(t)\right) \\ f_{i}(t) &=-h_{i}\left(X_{1}(t)\right) \beta V_{i}(t), \quad \text { for } i=2,3 \\ g_{i}(t) &=\sqrt{h_{i}\left(X_{1}(t)\right)} \beta \sqrt{2 D}, \quad \text { for } i=1,2,3 \end{aligned}
$$

where the functions $h_{k}:[-R, R] \rightarrow \mathbb{R}$ are given by

$$
\begin{array}{c}{h_{0}\left(x_{1}\right)=\frac{3 \beta \sigma_{\mu} \sqrt{\pi}}{8 \sqrt{2}}\left(1-\frac{x_{1}^{2}}{R^{2}}\right), \quad h_{1}\left(x_{1}\right)=\frac{1}{2}\left(1-\frac{x_{1}^{3}}{R^{3}}\right)} \\ {h_{2}\left(x_{1}\right)=h_{3}\left(x_{1}\right)=\frac{1}{4}\left(2-3 \frac{x_{1}}{R}+\frac{x_{1}^{3}}{R^{3}}\right)}\end{array}
$$

>Equation (3) can be used to design a multi-resolution scheme which couples the MD model with its corresponding Langevin description. Considering the boundry $\left\{x_{1}=0\right\},$ the multi-resolution scheme simply uses the MD model for $X_{1} \geq R$, the Langevin description for $X_{1} \leq-R$ and equation (3) for $X_{1} \in(-R, R)$. The other sides of the cube $\Omega$ are treated in a similar way. Technical details of such a multi-resolution scheme can be found in the paper by [Erban (2014)](https://royalsocietypublishing.org/doi/full/10.1098/rspa.2014.0036)

## Some Review Paper on Multi-Resolution Methods:

1. [Smith and Yates (2018)](https://royalsocietypublishing.org/doi/pdf/10.1098/rsif.2017.0931)

2. The decomposition of the computational domain into subdomains can also be made time-dependent. [Robinson et al. (2014)](https://aip.scitation.org/doi/full/10.1063/1.4868652)

3. Couple individual-based descriptions of cells with PDE models of their environment; i.e. both an individual-based description and the macroscopic PDEs are used in the whole computational domain, but they describe different components of the studied system; see, for example, [Franz and Erban (2012)](https://link.springer.com/chapter/10.1007/978-3-642-35497-7_5)